In [8]:
# import important library
import pandas as pd
import numpy as np
import bot

In [200]:
import importlib
importlib.reload(bot)

<module 'bot' from '/Users/henrytran/Documents/GitHub/aias_trading25/bot.py'>

# 1. Data Processing

In [10]:
# read the csv file
daily_df=pd.read_csv('BTC-Daily.csv')
daily_df.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
0,1646092800,2022-03-01 00:00:00,BTC/USD,43221.71,43626.49,43185.48,43185.48,49.006289,2.116360e+06
1,1646006400,2022-02-28 00:00:00,BTC/USD,37717.10,44256.08,37468.99,43178.98,3160.618070,1.364723e+08
2,1645920000,2022-02-27 00:00:00,BTC/USD,39146.66,39886.92,37015.74,37712.68,1701.817043,6.418008e+07
3,1645833600,2022-02-26 00:00:00,BTC/USD,39242.64,40330.99,38600.00,39146.66,912.724087,3.573010e+07
4,1645747200,2022-02-25 00:00:00,BTC/USD,38360.93,39727.97,38027.61,39231.64,2202.851827,8.642149e+07


In [11]:
# convert time strings into timestamps, and take only year, month and date values
daily_df["date"]=pd.to_datetime(daily_df["date"]).dt.strftime("%Y-%m-%d")

In [12]:
daily_df.shape

(2651, 9)

The dataset has 2651 records from 2014 to 2022.

As the data is needed for training and testing, every records before 2020 will be used for training and every records after 2020 will be used for testing.

In [130]:
# splitting the data
training = daily_df[daily_df["date"]<"2020-01-01"].iloc[::-1]
testing = daily_df[daily_df["date"]>="2020-01-01"].iloc[::-1]

In [131]:
# convert the data into csv files
training.to_csv('training.csv')
testing.to_csv('testing.csv')

After splitting, the training dataset has 1860 records and the testing data has 791 records.

In [15]:
print(training.shape)
print(testing.shape)

(1860, 9)
(791, 9)


# 2. Hill-climbing algorithm

## a. Try the bot

In [16]:
# extract the data
close_price_train=training["close"]
no_of_day_train=training.shape[0]
close_price_test =testing["close"]
no_of_day_test=testing.shape[0]

In [17]:
low_sma = bot.wma(close_price_train, 10, bot.sma_filter(10))
high_sma = bot.wma(close_price_train, 20, bot.sma_filter(20))

print(low_sma)
print(len(low_sma))
print(high_sma)
print(len(high_sma))

[-5844.114 -4371.12  -2900.949 ...   371.484   374.107   375.274]
1860
[-6466.681  -5746.248  -5014.0975 ...   354.9755   357.9115   361.117 ]
1860


In [18]:
# self-implementation for trading bot of sma
def bot_sma(close_price, hfw, lfw):
    low_sma = bot.wma(close_price, lfw, bot.sma_filter(lfw))
    high_sma = bot.wma(close_price, hfw, bot.sma_filter(hfw))
    buy_sell_signals=[]
    for i in range(len(low_sma)-1):
        if low_sma[i]<high_sma[i] and low_sma[i+1]>high_sma[i+1]:
            buy_sell_signals.append("buy")
        elif low_sma[i]>high_sma[i] and low_sma[i+1]<high_sma[i+1]:
            buy_sell_signals.append("sell")
        else:
            buy_sell_signals.append("-")
    return buy_sell_signals

## b. Hill-climbing algorithm

In [288]:
def hill_climbing(bot_type, high_frequency_window, low_frequency_window, alpha=0, max_iter=100):
    high_window = high_frequency_window
    low_window = low_frequency_window
    new_high_frequency_window = high_frequency_window
    new_low_frequency_window= low_frequency_window
    alpha=alpha
    new_alpha=alpha
    cash1=0
    for i in range(max_iter):
        # find the correct parameter tweak:
        if bot_type.lower() == 'sma':
            new_high_frequency_window, new_low_frequency_window = parameter_tweak('sma', high_window, low_window)
        elif bot_type.lower() == 'smaema' and alpha !=0:
            new_high_frequency_window, new_low_frequency_window, new_alpha = parameter_tweak('smaema', high_window, low_window, alpha)
        elif bot_type.lower() == 'complex':
            new_high_frequency_window, new_low_frequency_window = parameter_tweak('complex',high_window, low_window)
            
        # run the total cash return after trading
        cash1=bot_fitness_func(bot_type, high_window, low_window, alpha)
        cash2=bot_fitness_func(bot_type, new_high_frequency_window, new_low_frequency_window, new_alpha)
        # compare the cash earned after tweaking the parameters
        if cash2 > cash1:
            high_window = new_high_frequency_window
            low_window = new_low_frequency_window
            alpha=new_alpha
            cash1 = cash2
    if bot_type.lower() == 'sma' or bot_type.lower() == 'complex':
        return high_window, low_window, float(cash1)
    elif bot_type.lower() == 'smaema':
        return high_window, low_window, alpha, float(cash1)

def parameter_tweak(bot_type, hfw, lfw, alpha =0):
    if bot_type.lower() == 'sma':
        new_hfw, new_lfw = window_tweak(hfw, lfw)
        return new_hfw, new_lfw
    elif bot_type.lower() == 'smaema' and alpha !=0:
        new_hfw, new_lfw = window_tweak(hfw, lfw)
        new_alpha = alpha_tweak(alpha)
        return new_hfw, new_lfw, new_alpha
    elif bot_type.lower() == 'complex':
        new_hfw, new_lfw = complex_tweak(hfw, lfw)
        return new_hfw, new_lfw

def window_tweak(hfw, lfw):
    rng = np.random.default_rng()
    for _ in range(100):
        # new high frequency window
        a=int(rng.integers(-5,6))
        new_hfw=hfw+a
        # new low frequency window
        b=int(rng.integers(-5,6))
        new_lfw=lfw+b
        # we check to make sure that new_hfw in range(11,40) and new_lfw(2,10)
        new_hfw = max(2, max(2, new_hfw))
        new_lfw = max(11, min(40, new_lfw))
        if new_hfw < new_lfw:
            return new_hfw, new_lfw
    return hfw, lfw

def alpha_tweak(alpha):
    rng = np.random.default_rng()
    for _ in range(100):
        diff = rng.uniform(-0.15, 0.15)
        new_alpha = alpha + diff
        if 0.1<= new_alpha <= 1:
            return new_alpha
    return alpha # if after 100 loops and cannot find the optimal value, then we return the original alpha

def weights_tweak(weight_lst):
    rng = np.random.default_rng()
    # random index number of a weight in a list
    position=int(rng.integers(0,3))
    # normalised weights
    normalised_weights = [float(weight)/sum(weight_lst) for weight in weight_lst]

    # tweak the randomly selected weight with proportion different
    selected_weight_rate = normalised_weights[position]
    diff = np.random.uniform(-0.15, 0.15)
    selected_weight_rate += diff

    # the rate of other weights
    other_weight_idx= [idx for idx in range(3) if idx!=position]
    remainder_rate = 1- selected_weight_rate 
    # the remaining 2 weights, we generate randomly the proportion and multiply it with the current rates 
    one_weight_rate = rng.uniform(0.1, 0.9)
    another_weight_rate = 1- one_weight_rate
    other_weight_rates=[one_weight_rate, another_weight_rate]
    new_other_weight_rates = [remainder_rate * s for s in other_weight_rates]

    new_weights = [0,0,0]
    new_weights[position] = selected_weight_rate * sum(weight_lst)
    for i in range(2):
        idx = other_weight_idx[i]
        new_weights[idx] = new_other_weight_rates[i] * sum(weight_lst)

    return new_weights[0], new_weights[1], new_weights[2]


def complex_tweak(hfw, lfw):
    # in this code, hfw and lfw input are array-type in format of [w1, w2, w3, d1, d2, d3, sf]
    # weight tweak
    new_hfw_w1, new_hfw_w2,new_hfw_w3 = weights_tweak(hfw[0:3])
    new_lfw_w1, new_lfw_w2,new_lfw_w3 = weights_tweak(lfw[0:3])

    # window tweak
    new_hfw_d1, new_lfw_d1 = window_tweak(hfw[3], lfw[3])
    new_hfw_d2, new_lfw_d2 = window_tweak(hfw[4], lfw[4])
    new_hfw_d3, new_lfw_d3 = window_tweak(hfw[5], lfw[5])

    # alpha tweak
    new_hfw_alpha = alpha_tweak(hfw[-1])
    new_lfw_alpha = alpha_tweak(lfw[-1])

    return [new_hfw_w1, new_hfw_w2, new_hfw_w3, new_hfw_d1, new_hfw_d2, new_hfw_d3,new_hfw_alpha], [new_lfw_w1, new_lfw_w2, new_lfw_w3, new_lfw_d1, new_lfw_d2, new_lfw_d3, new_lfw_alpha]



In [292]:
#sma
hfw_result, lfw_result, cash_result = hill_climbing('sma',5,30)
hfw_result, lfw_result, cash_result

(12, 40, 7099.280588360062)

In [294]:
#smaema
hfw_result, lfw_result, alpha, cash_result = hill_climbing('smaema', 5, 30, 0.2)
hfw_result, lfw_result, alpha, cash_result

(15, 29, 0.26241341588934075, 7187.190558171788)

In [320]:
#complex
hfw_result, lfw_result, cash_result = hill_climbing('complex',[2,2,2,5,5,5,0.3],[2,2,2,30,30,30,0.3])
hfw_result, lfw_result, cash_result

([0.32912138708409844,
  3.7004331038042135,
  1.9704455091116895,
  5,
  4,
  11,
  0.3990979891147773],
 [2.978000542748241,
  0.7095151509177311,
  2.3124843063340283,
  17,
  34,
  31,
  0.4816705424185288],
 31899.36343769777)

# 3. Fitness function and Bot evaluation function

In [322]:
def bot_fitness_func(bot_type, high_window, low_window, alpha=0): # change bot_signals to high_frequency_window_size, low_frequency_window_size
    """
    This function will calculate the fitness (total cash earned from the buy/sell signals) of the trading bot
    Parameters:
    - 
    - 
    Return:
    - 
    """
    # intialise bot, use training dataset for optimisation algorithms fitness functions
    bot_signals=[]
    close_price = pd.read_csv('training.csv')['close']
    
    if bot_type.lower() == 'sma' and alpha ==0:
        bot_signals = bot.get_signals_sma2(close_price, high_window, low_window) #need to think about how to call 2 other bot algorithms
    elif bot_type.lower() == 'smaema' and alpha!=0:
        bot_signals = bot.get_signals_smaema(close_price, high_window, low_window, alpha)
    elif bot_type.lower() == 'complex' and alpha==0: #as alpha values were stored in the list of high_window and low_window
        bot_signals = bot.get_signals_complex(close_price, high_window, low_window)

    # initial values
    cash = 1000
    fee=0.03
    bitcoin = 0.0

    #loop through the time length
    for i in range(len(close_price)-1):
        close=close_price.iloc[i]
        # buy, ensure we have cash to buy
        if bot_signals[i] == "buy" and cash>0:
            bitcoin =  (cash*(1-fee))/close
            cash = 0
        # sell, ensure we have bitcoin to sell
        elif bot_signals[i] == "sell" and bitcoin>0:
            cash = bitcoin * close * (1-fee)
            bitcoin =0
    
    # final evaluation to change back to cash
    last_close=close_price.iloc[-1]
    if bitcoin>0:
        cash = bitcoin * last_close * (1-fee)
        bitcoin =0
        return cash
    elif cash >0:
        return cash

In [326]:
def bot_testing(bot_type, high_window, low_window, alpha=0):
    # This function will calculate the fitness (total cash earned from the buy/sell signals) of the trading bot, including the time of the transactions made
    # initial values
    cash = 1000
    fee=0.03
    bitcoin = 0.0

    # get the data
    close_price = pd.read_csv('testing.csv')['close']
    time = pd.read_csv('testing.csv')['date']

    # the list to save the transaction history
    result=[]
    bot_signals=[]

    # intialise bot, use training dataset for optimisation algorithms fitness functions
    if bot_type.lower() == 'sma' and alpha ==0:
        bot_signals = bot.get_signals_sma2(close_price, high_window, low_window)
    elif bot_type.lower() == 'smaema' and alpha!=0:
        bot_signals = bot.get_signals_smaema(close_price, high_window, low_window, alpha)
    elif bot_type.lower() == 'complex' and alpha==0: #as alpha values were stored in the list of high_window and low_window
        bot_signals = bot.get_signals_complex(close_price, high_window, low_window)
    
    #loop through the time length
    for i in range(time.shape[0]-1):
        close=close_price[i]
        # buy
        if bot_signals[i] == "buy" and cash>0:
            bitcoin =  (cash*(1-fee))/close
            cash = 0
            result.append([time[i],cash, bitcoin])
        # sell
        elif bot_signals[i] == "sell" and bitcoin>0:
            cash = bitcoin * close * (1-fee)
            bitcoin =0
            result.append([time[i],cash, bitcoin])
    
    # final evaluation to change back to cash
    last_close=close_price.iloc[-1]
    if bitcoin>0:
        cash = bitcoin * last_close * (1-fee)
        bitcoin =0
        result.append([time[i],cash, bitcoin])
        return result
    elif cash>0:
        return result

In [324]:
def bot_evaluation(bot_type, high_window, low_window, alpha=0):
    # This function will returns the result nicely
    result_lst=bot_testing(bot_type, high_window, low_window, alpha)
    result_df=pd.DataFrame(result_lst, columns=["Time", "Cash", "Bitcoin"])
    print(result_df.to_string(index=False, justify="center", float_format='{:,.2f}'.format))
    return result_df

In [293]:
#sma - nice printout
bot_evaluation('sma', hfw_result, lfw_result, alpha=0)

   Time      Cash    Bitcoin
2020-01-01     0.00   0.14  
2020-02-29 1,117.72   0.00  
2020-04-10     0.00   0.16  
2020-06-21 1,419.60   0.00  
2020-07-24     0.00   0.14  
2020-09-03 1,421.33   0.00  
2020-10-08     0.00   0.13  
2021-01-31 4,056.75   0.00  
2021-02-05     0.00   0.10  
2021-04-24 4,992.27   0.00  
2021-07-28     0.00   0.12  
2021-09-16 5,603.48   0.00  
2021-10-07     0.00   0.10  
2021-11-21 5,751.72   0.00  
2022-02-11     0.00   0.13  
2022-02-26 4,995.82   0.00  


,Time,Cash,Bitcoin
0,2020-01-01,0.000000,0.135122
1,2020-02-29,1117.719492,0.000000
2,2020-04-10,0.000000,0.157654
3,2020-06-21,1419.597182,0.000000
4,2020-07-24,0.000000,0.144221
5,2020-09-03,1421.326942,0.000000
6,2020-10-08,0.000000,0.126193
7,2021-01-31,4056.753777,0.000000
8,2021-02-05,0.000000,0.102689
9,2021-04-24,4992.274932,0.000000


In [295]:
#smaema
bot_evaluation('smaema',hfw_result, lfw_result, alpha)

   Time      Cash    Bitcoin
2020-01-01     0.00   0.14  
2020-02-24 1,265.15   0.00  
2020-04-04     0.00   0.18  
2020-05-25 1,540.58   0.00  
2020-05-30     0.00   0.15  
2020-06-13 1,414.94   0.00  
2020-07-22     0.00   0.14  
2020-08-25 1,581.04   0.00  
2020-09-30     0.00   0.14  
2020-10-02 1,460.04   0.00  
2020-10-09     0.00   0.13  
2021-01-23 3,992.16   0.00  
2021-02-05     0.00   0.10  
2021-03-05 4,777.55   0.00  
2021-03-08     0.00   0.09  
2021-04-07 4,795.72   0.00  
2021-04-08     0.00   0.08  
2021-04-20 4,388.53   0.00  
2021-05-09     0.00   0.07  
2021-05-10 3,957.47   0.00  
2021-06-15     0.00   0.10  
2021-06-19 3,291.22   0.00  
2021-07-26     0.00   0.09  
2021-09-09 3,852.52   0.00  
2021-10-04     0.00   0.08  
2021-11-16 4,427.22   0.00  
2022-02-07     0.00   0.10  
2022-02-20 3,643.79   0.00  


,Time,Cash,Bitcoin
0,2020-01-01,0.000000,0.135122
1,2020-02-24,1265.150769,0.000000
2,2020-04-04,0.000000,0.178382
3,2020-05-25,1540.584851,0.000000
4,2020-05-30,0.000000,0.153978
5,2020-06-13,1414.938952,0.000000
6,2020-07-22,0.000000,0.143874
7,2020-08-25,1581.037169,0.000000
8,2020-09-30,0.000000,0.142288
9,2020-10-02,1460.038262,0.000000


In [327]:
#complex
bot_evaluation('complex',hfw_result, lfw_result)

   Time      Cash    Bitcoin
2020-03-21     0.00   0.16  
2021-05-19 5,592.75   0.00  
2021-05-26     0.00   0.14  
2022-02-28 5,781.07   0.00  


,Time,Cash,Bitcoin
0,2020-03-21,0.000000,0.156761
1,2021-05-19,5592.749999,0.000000
2,2021-05-26,0.000000,0.138006
3,2022-02-28,5781.072865,0.000000
